In [71]:
import pandas as pd
import numpy as np
import collections
import sys
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import linear_model
from sklearn import svm
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import csv

In [2]:
## Read in dataframes 
data_dir = './DataFiles/'
# This file provides a master list of cities that have been locations for games played.
df_cities = pd.read_csv(data_dir + 'Cities.csv')
df_conferences = pd.read_csv(data_dir + 'Conferences.csv')
df_conferencetourney = pd.read_csv(data_dir + 'ConferenceTourneyGames.csv')
df_gamecities = pd.read_csv(data_dir + 'GameCities.csv')
df_tourneycompact = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_tourneydetailed = pd.read_csv(data_dir + 'NCAATourneyDetailedResults.csv')
df_tourneyseedroundslots = pd.read_csv(data_dir + 'NCAATourneySeedRoundSlots.csv')
df_tourneyseeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tourneyslots = pd.read_csv(data_dir + 'NCAATourneySlots.csv')
df_seasoncompact = pd.read_csv(data_dir + 'RegularSeasonCompactResults.csv')
df_seasondetailed = pd.read_csv(data_dir + 'RegularSeasonDetailedResults.csv')
df_seasons = pd.read_csv(data_dir + 'Seasons.csv')
df_secondtourneycompact = pd.read_csv(data_dir + 'SecondaryTourneyCompactResults.csv')
df_coaches = pd.read_csv(data_dir + 'TeamCoaches.csv')
df_teamconferences = pd.read_csv(data_dir + 'TeamConferences.csv')
df_teams = pd.read_csv(data_dir + 'Teams.csv')
# df_teamspellings = pd.read_csv(data_dir + 'TeamSpellings.csv') utf-8 encoding issue

df_teams.head()

TeamID     TeamName  FirstD1Season  LastD1Season
0    1101  Abilene Chr           2014          2018
1    1102    Air Force           1985          2018
2    1103        Akron           1985          2018
3    1104      Alabama           1985          2018
4    1105  Alabama A&M           2000          2018

In [3]:
def getTeamName(team_id):
    return df_teams[df_teams['TeamID'] == team_id].values[0][1]

getTeamName(1200)

'Fordham'

In [4]:
df_year = df_seasoncompact[df_seasoncompact['Season'] == 2010]
df_year2 = df_seasondetailed[df_seasondetailed['Season'] == 2010]
#df_year.info()
gamesWon = df_year[df_year.WTeamID == 1120] 
gamesWon2 = df_year2[df_year2.WTeamID == 1120] 
print(gamesWon.head(10))
#print(gamesWon2.head(10))

        Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
108134    2010      11     1120      69     1310      65    H      0
108535    2010      19     1120      80     1237      65    N      0
108770    2010      23     1120      75     1219      54    H      0
109033    2010      29     1120      87     1105      52    N      0
109322    2010      35     1120      68     1438      67    H      0
109795    2010      50     1120      94     1106      78    H      0
109976    2010      57     1120      77     1149      62    H      0
110135    2010      61     1120      95     1204      75    H      0
110991    2010      79     1120      84     1261      80    A      0
111458    2010      89     1120      58     1104      57    H      0


In [5]:
tourneyYear = df_tourneyseeds[df_tourneyseeds['Season'] == 2010]
seed = tourneyYear[tourneyYear['TeamID'] == 1246]
if (len(seed.index) != 0):
    seed = seed.values[0][1]
    tournamentSeed = int(seed[1:3])
    
tournamentSeed

1

In [6]:
def getSeasonData(team_id, year):
    # The data frame below holds stats for every single game in the given year
    df_year = df_seasondetailed[df_seasondetailed['Season'] == year]
    # Finding number of points per game
    gamesWon = df_year[df_year.WTeamID == team_id] 
    totalPoints = gamesWon['WScore'].sum()
    gamesLost = df_year[df_year.LTeamID == team_id] 
    totalGames = gamesWon.append(gamesLost)
    numGames = len(totalGames.index)
    totalPoints += gamesLost['LScore'].sum()
    
    # Finding number of points per game allowed
    totalPointsAllowed = gamesWon['LScore'].sum()
    totalPointsAllowed += gamesLost['WScore'].sum()
    
    #FGM - field goals made
    totalFGM = gamesWon['WFGM'].sum()
    totalFGM += gamesLost['LFGM'].sum()
    
    #FGM - field goals made (by opponent)
    totalOppFGM = gamesWon['LFGM'].sum()
    totalOppFGM += gamesLost['WFGM'].sum()
    
    #FGA - field goals attempted
    totalFGA = gamesWon['WFGA'].sum()
    totalFGA += gamesLost['LFGA'].sum()
    
    #FGA - field goals attempted (by opponent)
    totalOppFGA = gamesWon['LFGA'].sum()
    totalOppFGA += gamesLost['WFGA'].sum()
    
    #FGM3 - three pointers made
    totalFGM3 = gamesWon['WFGM3'].sum()
    totalFGM3 += gamesLost['LFGM3'].sum()
    
    #FGM3 - three pointers made (by opponent)
    totalOppFGM3 = gamesWon['LFGM3'].sum()
    totalOppFGM3 += gamesLost['WFGM3'].sum()
    
    #FGA3 - three pointers attempted
    totalFGA3 = gamesWon['WFGA3'].sum()
    totalFGA3 += gamesLost['LFGA3'].sum()
    
    #FGA3 - three pointers attempted (by opponent)
    totalOppFGA3 = gamesWon['LFGA3'].sum()
    totalOppFGA3 += gamesLost['WFGA3'].sum()
    
    #FTM - free throws made
    totalFTM = gamesWon['WFTM'].sum()
    totalFTM += gamesLost['LFTM'].sum()
    
    #FTM - free throws made (by opponent)
    totalOppFTM = gamesWon['LFTM'].sum()
    totalOppFTM += gamesLost['WFTM'].sum()
    
    #FTA - free throws attempted
    totalFTA = gamesWon['WFTA'].sum()
    totalFTA += gamesLost['LFTA'].sum()
    
    #FTA - free throws attempted (by opponent)
    totalOppFTA = gamesWon['LFTA'].sum()
    totalOppFTA += gamesLost['WFTA'].sum()
    
    #OR - offensive rebounds
    totalOR = gamesWon['WOR'].sum()
    totalOR += gamesLost['LOR'].sum()
    
    #OR - offensive rebounds (by opponent)
    totalOppOR = gamesWon['LOR'].sum()
    totalOppOR += gamesLost['WOR'].sum()
    
    #DR - defensive rebounds
    totalDR = gamesWon['WDR'].sum()
    totalDR += gamesLost['LDR'].sum()
    
    #DR - defensive rebounds (by opponent)
    totalOppDR = gamesWon['LDR'].sum()
    totalOppDR += gamesLost['WDR'].sum()
    
    #Ast - assists
    totalAst = gamesWon['WAst'].sum()
    totalAst += gamesLost['LAst'].sum()
    
    #Ast - assists (by opponent)
    totalOppAst = gamesWon['LAst'].sum()
    totalOppAst += gamesLost['WAst'].sum()
    
    #TO - turnovers committed
    totalTO = gamesWon['WTO'].sum()
    totalTO += gamesLost['LTO'].sum()
    
    #TO - turnovers committed (by opponent)
    totalOppTO = gamesWon['LTO'].sum()
    totalOppTO += gamesLost['WTO'].sum()
    
    #Stl - steals
    totalStl = gamesWon['WStl'].sum()
    totalStl += gamesLost['LStl'].sum()
    
    #Stl - steals (by opponent)
    totalOppStl = gamesWon['LStl'].sum()
    totalOppStl += gamesLost['WStl'].sum()
    
    #Blk - blocks
    totalBlk = gamesWon['WBlk'].sum()
    totalBlk += gamesLost['LBlk'].sum()
    
    #Blk - blocks (by opponent)
    totalOppBlk = gamesWon['LBlk'].sum()
    totalOppBlk += gamesLost['WBlk'].sum()
    
    #PF - personal fouls committed
    totalPF = gamesWon['WPF'].sum()
    totalPF += gamesLost['LPF'].sum()
    
    #PF - personal fouls committed (by opponent)
    totalOppPF = gamesWon['LPF'].sum()
    totalOppPF += gamesLost['WPF'].sum()
    
    #Finding tournament seed for that year
    tourneyYear = df_tourneyseeds[df_tourneyseeds['Season'] == year]
    seed = tourneyYear[tourneyYear['TeamID'] == team_id]
    if (len(seed.index) != 0):
        seed = seed.values[0][1]
        tournamentSeed = int(seed[1:3])
    else:
        tournamentSeed = 25
    
    # Finding number of wins and losses
    numWins = len(gamesWon.index)
    
    # Creating averages
    if numGames == 0:
        avgPoints = 0
        avgPointsAllowed = 0    
        avgFGM = 0
        avgOppFGM = 0
        avgFGA = 0
        avgOppFGA = 0
        avgFGM3 = 0
        avgOppFGM3 = 0
        avgFGA3 = 0
        avgOppFGA3 = 0
        avgFTM = 0
        avgOppFTM = 0
        avgFTA = 0
        avgOppFTA = 0
        avgOR = 0
        avgOppOR = 0
        avgDR = 0
        avgOppDR = 0
        avgAst = 0
        avgOppAst = 0
        avgTO = 0
        avgOppTO = 0
        avgStl = 0
        avgOppStl = 0
        avgBlk = 0
        avgOppBlk = 0
        avgPF = 0
        avgOppPF = 0
        tournamentSeed = 0
    else:
        avgPoints = totalPoints/numGames
        avgPointsAllowed = totalPointsAllowed/numGames
        avgFGM = totalFGM/numGames
        avgOppFGM = totalOppFGM/numGames
        avgFGA = totalFGA/numGames
        avgOppFGA = totalOppFGA/numGames
        avgFGM3 = totalFGM3/numGames
        avgOppFGM3 = totalOppFGM3/numGames
        avgFGA3 = totalFGA3/numGames
        avgOppFGA3 = totalOppFGA3/numGames
        avgFTM = totalFTM/numGames
        avgOppFTM = totalOppFTM/numGames
        avgFTA = totalFTA/numGames
        avgOppFTA = totalOppFTA/numGames
        avgOR = totalOR/numGames
        avgOppOR = totalOppOR/numGames
        avgDR = totalDR/numGames
        avgOppDR = totalOppDR/numGames
        avgAst = totalAst/numGames
        avgOppAst = totalOppAst/numGames
        avgTO = totalTO/numGames
        avgOppTO = totalOppTO/numGames
        avgStl = totalStl/numGames
        avgOppStl = totalOppStl/numGames
        avgBlk = totalBlk/numGames
        avgOppBlk = totalOppBlk/numGames
        avgPF = totalPF/numGames
        avgOppPF = totalOppPF/numGames

    #return gamesLost
    return [numWins, avgPoints, avgPointsAllowed, avgFGM, avgOppFGM, avgFGA, avgOppFGA, avgFGM3, avgOppFGM3, avgFGA3, avgOppFGA3,
            avgFTM, avgOppFTM, avgFTA, avgOppFTA, avgOR, avgOppOR, avgDR, avgOppDR, avgAst, avgOppAst, avgTO, avgOppTO, avgStl,
            avgOppStl, avgBlk, avgOppBlk, avgPF, avgOppPF, tournamentSeed]
    
    #return [numWins]

In [7]:
#getSeasonData(1246, 2010) #Kentucky

In [8]:
#getSeasonData(1242, 2016) #Kansas

In [9]:
def compareTwoTeams(id_1, id_2, year):
    team_1 = getSeasonData(id_1, year)
    team_2 = getSeasonData(id_2, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [10]:
compareTwoTeams(1246,1452, 2010)

[5,
 6.265597147950089,
 1.476827094474153,
 2.7272727272727266,
 1.4429590017825333,
 0.22816399286987377,
 9.62566844919786,
 -0.9688057040998217,
 1.1639928698752229,
 -3.166666666666668,
 4.491087344028518,
 1.7798573975044576,
 -2.573083778966131,
 3.1987522281639933,
 -4.0,
 -1.2816399286987519,
 2.519607843137255,
 4.301247771836007,
 -0.5044563279857393,
 -0.9910873440285215,
 -0.5445632798573978,
 2.7727272727272734,
 0.5427807486631018,
 1.6586452762923347,
 0.9135472370766484,
 3.235294117647059,
 0.5418894830659537,
 -2.098039215686274,
 0.0026737967914414185,
 -1]

In [11]:
teamList = df_teams['TeamName'].tolist()

def createSeasonDict(year):
    seasonDictionary = collections.defaultdict(list)
    for team in teamList:
        team_id = df_teams[df_teams['TeamName'] == team].values[0][0]
        team_vector = getSeasonData(team_id, year)
        seasonDictionary[team_id] = team_vector
    return seasonDictionary


In [12]:
def getHomeStat(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = -1
    if (row == 'N'):
        home = 0
    return home

In [13]:
df_seasoncompact.head()

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985      20     1228      81     1328      64    N      0
1    1985      25     1106      77     1354      70    H      0
2    1985      25     1112      63     1223      56    H      0
3    1985      25     1165      70     1432      54    H      0
4    1985      25     1192      86     1447      74    H      0

In [14]:
def createTrainingSet(years):
    totalNumGames = 0
    for year in years:
        season = df_seasoncompact[df_seasoncompact['Season'] == year]
        totalNumGames += len(season.index)
        tourney = df_tourneycompact[df_tourneycompact['Season'] == year]
        totalNumGames += len(tourney.index)
    numFeatures = len(getSeasonData(1246,2010))
    #xTrain = np.zeros((totalNumGames, numFeatures + 1))
    #yTrain = np.zeros((totalNumGames))
    xTrain=[]
    yTrain=[]
    indexCounter = 0
    for year in years:
        team_vectors = createSeasonDict(year)
        season = df_seasoncompact[df_seasoncompact['Season'] == year]
        numGamesInSeason = len(season.index)
        tourney = df_tourneycompact[df_tourneycompact['Season'] == year]
        numGamesInSeason += len(tourney.index)
        #xTrainSeason = np.zeros(( numGamesInSeason, numFeatures + 1))
        #yTrainSeason = np.zeros(( numGamesInSeason ))
        xTrainSeason = []
        yTrainSeason = []
        counter = 0
        for index, row in season.iterrows():
            w_team = row['WTeamID']
            w_vector = team_vectors[w_team]
            #if w_vector is np.any(w_vector): #check if w_vector contains only zeros
            #    continue
            if all([x == 0 for x in w_vector]):
                continue
            l_team = row['LTeamID']
            l_vector = team_vectors[l_team]
            diff = [a - b for a, b in zip(w_vector, l_vector)]
            home = getHomeStat(row['WLoc'])
            if (counter % 2 == 0):
                diff.append(home) 
                #xTrainSeason[counter] = diff
                #yTrainSeason[counter] = 1
                xTrainSeason.append(diff)
                yTrainSeason.append(1)
            else:
                diff.append(-home)
                #xTrainSeason[counter] = [ -p for p in diff]
                #yTrainSeason[counter] = 0
                xTrainSeason.append([-p for p in diff])
                yTrainSeason.append(0)
            counter += 1
        for index, row in tourney.iterrows():
            w_team = row['WTeamID']
            w_vector = team_vectors[w_team]
            #if w_vector is np.any(w_vector): #check if w_vector contains only zeros
            #    continue
            if all([x == 0 for x in w_vector]):
                continue
            l_team = row['LTeamID']
            l_vector = team_vectors[l_team]
            diff = [a - b for a, b in zip(w_vector, l_vector)]
            home = 0 #All tournament games are neutral
            if (counter % 2 == 0):
                diff.append(home) 
                #xTrainSeason[counter] = diff
                #yTrainSeason[counter] = 1
                xTrainSeason.append(diff)
                yTrainSeason.append(1)
            else:
                diff.append(-home)
                #xTrainSeason[counter] = [ -p for p in diff]
                #yTrainSeason[counter] = 0
                xTrainSeason.append([-p for p in diff])
                yTrainSeason.append(0)
            counter += 1
        xTrain[indexCounter:numGamesInSeason+indexCounter] = xTrainSeason
        yTrain[indexCounter:numGamesInSeason+indexCounter] = yTrainSeason
        indexCounter += numGamesInSeason
    #xTrain[(xTrain != 0).sum(axis=1) >= 5, :]
    #xTrain[:,:29][~np.all(xTrain == 0, axis=1)]
    xTrain = np.array(xTrain)
    yTrain = np.array(yTrain)
    return xTrain, yTrain

## Testing

In [153]:
numFeatures = len(getSeasonData(1246,2010))
print(numFeatures)
team_vectors = createSeasonDict(2016)
season = df_seasoncompact[df_seasoncompact['Season'] == 2016]
numGamesInSeason = len(season.index)
tourney = df_tourneycompact[df_tourneycompact['Season'] == 2016]
numGamesInSeason += len(tourney.index)
xTrainSeason = np.zeros(( numGamesInSeason, numFeatures + 1))
yTrainSeason = np.zeros(( numGamesInSeason ))
counter = 0
for index, row in season.iterrows():
    w_team = row['WTeamID']
    w_vector = team_vectors[w_team]
    l_team = row['LTeamID']
    l_vector = team_vectors[l_team]
    diff = [a - b for a, b in zip(w_vector, l_vector)]
    home = getHomeStat(row['WLoc'])
    if (counter % 2 == 0):
        diff.append(home) 
        xTrainSeason[counter] = diff
        yTrainSeason[counter] = 1
    else:
        diff.append(-home)
        xTrainSeason[counter] = [-p for p in diff]
        yTrainSeason[counter] = 0
    counter += 1

#np.savetxt("xTrainSeason2016.csv", xTrainSeason, delimiter=",")
print(season.iloc[2,:])
print(team_vectors[1112])
print(team_vectors[1334])
diff = [a - b for a, b in zip(team_vectors[1112], team_vectors[1334])]
print(diff)
print(xTrainSeason[2])

30
Season     2016
DayNum       11
WTeamID    1112
WScore       79
LTeamID    1334
LScore       61
WLoc          H
NumOT         0
Name: 139922, dtype: object
[25, 81.21212121212122, 69.0, 28.060606060606062, 25.12121212121212, 58.24242424242424, 60.81818181818182, 6.515151515151516, 6.090909090909091, 17.03030303030303, 18.96969696969697, 18.575757575757574, 12.666666666666666, 25.696969696969695, 17.78787878787879, 11.636363636363637, 9.484848484848484, 28.606060606060606, 21.545454545454547, 14.515151515151516, 12.212121212121213, 12.818181818181818, 11.606060606060606, 4.909090909090909, 5.393939393939394, 4.787878787878788, 3.4545454545454546, 18.21212121212121, 22.0, 6]
[6, 68.5, 74.42307692307692, 23.807692307692307, 23.923076923076923, 55.34615384615385, 55.19230769230769, 6.576923076923077, 7.423076923076923, 20.423076923076923, 20.26923076923077, 14.307692307692308, 19.153846153846153, 21.26923076923077, 27.192307692307693, 10.038461538461538, 10.461538461538462, 24.923076923

In [154]:
years = range(2010,2013)
xTrain, yTrain = createTrainingSet(years)
#np.save('xTrain', xTrain)
#np.save('yTrain', yTrain)
print(xTrain)

[[ -5.           1.87594697   7.36268939 ...  -1.35606061  -5.
    1.        ]
 [-10.          -7.83266129   4.6875     ...  -1.75100806   0.
    1.        ]
 [ 25.          13.59013283 -20.79411765 ...  -2.26091082 -23.
    1.        ]
 ...
 [  0.          -0.26559715  -2.88057041 ...  -0.28342246   0.
    0.        ]
 [  6.           7.84789916  -2.17142857 ...  -0.73445378  -3.
    0.        ]
 [ -5.          -1.64616756   2.93939394 ...   0.34224599   1.
    0.        ]]


In [142]:
xTrain.shape, yTrain.shape
xTrain
#np.savetxt("xTrain.csv", xTrain, delimiter=",")
#np.savetxt("yTrain.csv", yTrain, delimiter=",")

In [204]:
xTrain.shape

(115113, 31)

In [208]:
x = createSeasonDict(2016)

In [219]:
#x[1109] - zeros
#1101 - not zeros
#print(x)
all([y == 0 for y in x[1109]])

True

## End of Testing

In [15]:
years = range(1993,2017)
xTrain, yTrain = createTrainingSet(years)
#np.save('xTrain', xTrain)
#np.save('yTrain', yTrain)
#print(xTrain)
#xTrainA = np.array(xTrain)
#yTrainA = np.array(yTrain)
#print(yTrainA.shape)

In [72]:
#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.LogisticRegression()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = AdaBoostClassifier(n_estimators=100)
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#neuralNetwork(10)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)

In [73]:
def showDependency(predictions, test, stat, my_categories):
    difference = test[:,my_categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Probability of Team 1 Win')
    plt.xlabel(stat + ' Difference (Team 1 - Team 2)')
    plt.show()

In [74]:
def showFeatureImportance(my_categories):
    fx_imp = pd.Series(model.feature_importances_, index=my_categories)
    fx_imp /= fx_imp.max()
    fx_imp.sort()
    fx_imp.plot(kind='barh')

In [75]:
accuracy=[]
for i in range(1):
    X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == Y_test))
    #accuracy.append(np.mean(predictions == Y_test))
    print("Finished iteration:", i)
print("The accuracy is", sum(accuracy)/len(accuracy))

KeyboardInterrupt: 

In [48]:
accuracy=[]
for i in range(5):
    X_train, X_test, Y_train, Y_test = train_test_split(StandardScaler().fit_transform(xTrain), yTrain, test_size=0.33, random_state=42)
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == Y_test))
    #accuracy.append(np.mean(predictions == Y_test))
    print("Finished iteration:", i)
print("The accuracy is", sum(accuracy)/len(accuracy))

Finished iteration: 0
Finished iteration: 1
Finished iteration: 2
Finished iteration: 3
Finished iteration: 4
The accuracy is 0.7476062489501092


In [59]:
X_train, Y_train = shuffle(xTrain, yTrain)
#print(X_train[1,:])
#from sklearn.preprocessing import StandardScaler
#X_train, X_test, Y_train, Y_test = train_test_split(StandardScaler().fit_transform(xTrain), yTrain, test_size=0.2, random_state=42)
#x_max = X_train.max()
#x_min = X_train.min()
#X_train = X_train / X_train.max(axis=0)
#print(x_max)
#X_train = ( X_train - x_min ) / ( x_max - x_min + 1e-14)
#X_test = ( X_test - x_min ) / ( x_max - x_min + 1e-14)

logreg = linear_model.LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, Y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.5098, with best C: 1000.0


In [44]:
#test = pd.DataFrame(X_train)
#test.to_csv('./Xtrain4.csv')
X_train[9,:]
#xTrain[:,:29][~np.all(xTrain == 0, axis=1)]
#xTrain.shape, yTrain.shape


array([-1.27097985e-03,  7.49259630e-01,  6.24119136e-01,  5.20082178e-01,
        1.00228462e+00,  4.50271216e-01,  1.36113864e+00, -2.21536546e-01,
       -1.50020766e-01,  5.08957290e-01, -3.36351466e-01,  1.11104600e+00,
       -3.79816527e-01,  7.51268267e-01, -5.30972909e-01,  3.19262971e-01,
        1.08071391e+00,  3.41430784e-01, -9.09965827e-02,  8.69142093e-01,
        9.31499481e-01, -5.69721213e-02, -3.62563356e-01, -3.93606768e-01,
        2.89585189e-01,  1.40859192e+00,  8.79235631e-03, -5.60286382e-01,
        1.19291852e+00, -1.72516497e-03,  7.94086407e-01])

# Applying Model

In [54]:
def predictGame(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    #return model.predict([diff]) 
    return clf.predict_proba([diff])

In [60]:
# This was the national championship matchup last year
team1_name = 'North Carolina' #1314
team2_name = 'Villanova' #1437
team1_vector = getSeasonData(df_teams[df_teams['TeamName'] == team1_name].values[0][0], 2016)
team2_vector = getSeasonData(df_teams[df_teams['TeamName'] == team2_name].values[0][0], 2016)
print('Probability that ' + team1_name + ' wins:', predictGame(team1_vector, team2_vector, 0))
#team1_vector
#team2_vector
diff = [a - b for a, b in zip(team1_vector, team2_vector)]
diff
diff.append(0)
clf.predict_proba([diff])

Probability that North Carolina wins: [[0.53856157 0.46143843]]


array([[0.53856157, 0.46143843]])

In [61]:
sample_sub_pd = pd.read_csv('SampleSubmissionStage1.csv')
def createPrediction():
    results = [[0 for x in range(2)] for x in range(len(sample_sub_pd.index))]
    for index, row in sample_sub_pd.iterrows():
        matchup_id = row['ID']
        year = matchup_id[0:4]
        team1_id = matchup_id[5:9]
        team2_id = matchup_id[10:14]
        team1_vector = getSeasonData(int(team1_id), int(year))
        team2_vector = getSeasonData(int(team2_id), int(year))
        pred = predictGame(team1_vector, team2_vector, 0)
        results[index][0] = matchup_id
        results[index][1] = np.clip(pred[0,1], 0.05, 0.95)
        #results[index][1] = pred[0][1]
    results = pd.np.array(results)
    firstRow = [[0 for x in range(2)] for x in range(1)]
    firstRow[0][0] = 'id'
    firstRow[0][1] = 'pred'
    with open("result6.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(firstRow)
        writer.writerows(results)

In [62]:
createPrediction()

In [121]:
def getAllTeamVectors():
    year = 2016
    numFeatures = len(getSeasonData(1181,2012))
    numTeams = len(df_teams)
    teamvecs = np.zeros(( numTeams, numFeatures ))
    teams=[]
    counter = 0
    for team in teamList:
        team_id = df_teams[df_teams['TeamName'] == team].values[0][0]
        team_vector = getSeasonData(team_id, year)
        if (team_vector[0] == 0 or team_vector[4] == 0):
            continue
        teamvecs[counter] = team_vector
        teams.append(team)
        counter += 1
    team = pd.np.array(teams)
    team = np.reshape(team, (team.shape[0], 1))
    
    with open("allNames.csv", "w") as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerows(team)
    with open("allVecs.csv", "w") as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerows(teamvecs)
    with open("combine.csv", "w") as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerows([team,teamvecs])
getAllTeamVectors()